# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import pandas as pd
import numpy as np
import re

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admission_predict`
* table: `admission_predict`

In [2]:
path_file = '../dataset/Admission_Predict.csv'
adm_df = pd.read_csv(path_file)

Remove trailing spaces at the end of the column names if there are any.


In [3]:
# your code here
col_dict = {key:key.strip() for key in adm_df.columns.values}
adm_df.rename(columns=col_dict,inplace=True)


Let's evaluate the dataset by looking at the `head` function.

In [4]:
# your code here
adm_df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
# your code here
adm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         385 non-null    int64  
 1   GRE Score          385 non-null    int64  
 2   TOEFL Score        385 non-null    int64  
 3   University Rating  385 non-null    int64  
 4   SOP                385 non-null    float64
 5   LOR                385 non-null    float64
 6   CGPA               385 non-null    float64
 7   Research           385 non-null    int64  
 8   Chance of Admit    385 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 27.2 KB


Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [6]:
# your code here
adm_df.set_index('Serial No.', drop=False, inplace=True)

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [7]:
# your code here
zip_CGPA_GRE = [(gre,cgpa) for gre,cgpa in zip(adm_df['GRE Score'],adm_df.CGPA)]
unique_CGPA_GRE = set(zip_CGPA_GRE)
print(f"Check uniqueness GRE-CGPA, total length {len(adm_df)} and unique length: {len(unique_CGPA_GRE)}")

Check uniqueness GRE-CGPA, total length 385 and unique length: 385


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [8]:
adm_df.set_index(['GRE Score','CGPA'], drop=True, inplace=True)
adm_df.head()

,,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,,
337,9.65,1,118,4,4.5,4.5,1,0.92
316,8.00,2,104,3,3.0,3.5,1,0.72
322,8.67,3,110,3,3.5,2.5,1,0.80
314,8.21,4,103,2,2.0,3.0,0,0.65
330,9.34,5,115,5,4.5,3.0,1,0.90


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [9]:
# your code here
adm_df.reset_index(inplace=True)
adm_df.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [10]:
# your code here
cgpa_logic = adm_df.CGPA > 9
research_logic = adm_df.Research == 1
print(f"Number of students with CGPA > 9 and research experience: {(cgpa_logic & research_logic).sum()}")

Number of students with CGPA > 9 and research experience: 101


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [11]:
# your code here
sop_logic = adm_df.SOP < 3.5
print(f"Number of students with CGPA > 9 and SOP < 3.5: {(cgpa_logic & sop_logic).sum()}")
print(f"Mean chance of admit for these applicants: {round(adm_df['Chance of Admit'][cgpa_logic & sop_logic].mean()*100,2)} %")

Number of students with CGPA > 9 and SOP < 3.5: 5
Mean chance of admit for these applicants: 80.2 %


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [12]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    return (col - col.mean())/col.std()
    
    # your code here

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [13]:
# your code here
adm_df[['CGPA_std','GRE_std','LOR_std']] = adm_df[['CGPA','GRE Score','LOR']].apply(standardize)

We will generate the decision choice at random using the code below. Please run the cell.

In [14]:
# Libraries
from random import choices

In [15]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=adm_df.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [16]:
# your code here
adm_df['deciding_column'] = adm_df.lookup(adm_df.index,decision_choice)

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [17]:
# your code here
adm_df['decision'] = np.where(adm_df.deciding_column > 0.8, 1, 0)

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [18]:
# your code here
print(f"Total number of accepted applicants: {adm_df.decision.sum()}")

Total number of accepted applicants: 86


# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [19]:
# your code here
def modColNam(s):
    mod_str = re.sub(r'[^a-zA-Z0-9_\s]', '', s).lower()
    return re.sub(r'\s', '_', mod_str)
col_dict = {key:modColNam(key) for key in adm_df.columns.values}
col_dict
adm_df.rename(columns=col_dict,inplace=True)

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [20]:
# your code here
adjusted_gre = np.where(adm_df.university_rating >= 4, adm_df.gre_score+10, adm_df.gre_score)
adm_df['adjusted_gre'] = pd.cut(adjusted_gre, bins=4, labels=False)
adm_df.head()

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,1.747900,1,3
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,0.076740,0,1
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,0.455704,0,2
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,-0.480713,0,1
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,-0.480713,0,3
